In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/TransferLearning/InceptionV3
!ls

/content/gdrive/My Drive/TransferLearning/InceptionV3
X_test_inc.pickle  X_train_inc.pickle  Y_test_inc.pickle  Y_train_inc.pickle


In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import pickle

pickle_in = open("X_test_inc.pickle","rb")
X_test = pickle.load(pickle_in)

pickle_in = open("X_train_inc.pickle","rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_test_inc.pickle","rb")
Y_test = pickle.load(pickle_in)

pickle_in = open("Y_train_inc.pickle","rb")
Y_train = pickle.load(pickle_in)

In [5]:
X_train.shape

(1006, 299, 299, 3)

In [6]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
import keras.optimizers
from sklearn.metrics import classification_report
import keras.optimizers
from keras.applications import InceptionV3

Using TensorFlow backend.


In [0]:
tensorboard = TensorBoard(log_dir='./InceptionV3logs', histogram_freq=0,
                          write_graph=True, write_images=False)
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)


In [8]:
InceptionV3_model =  InceptionV3()
InceptionV3_model.summary()

type(InceptionV3_model)














Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________

keras.engine.training.Model

In [0]:
InceptionV3_model.layers.pop()
model = Sequential()
model.add(InceptionV3_model)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1000)              21802784  
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________________________________________


In [11]:
#model.layers.pop()
for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(4,activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 1000)              21802784  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 4004      
Total params: 21,806,788
Trainable params: 4,004
Non-trainable params: 21,802,784
_________________________________________________________________


In [13]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer= "adam",
              metrics=['accuracy'],
              )

In [14]:
#Fit the model
            model.fit(X_train, Y_train,
            batch_size=32,
            epochs=20,
            validation_data=(X_test,Y_test),
            callbacks=[tensorboard])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1006 samples, validate on 607 samples


Epoch 1/20
1006/1006 [==============================] - 31s 31ms/step - loss: 1.3850 - acc: 0.3757 - val_loss: 1.3835 - val_acc: 0.2768

Epoch 2/20
1006/1006 [==============================] - 16s 16ms/step - loss: 1.3816 - acc: 0.5189 - val_loss: 1.3832 - val_acc: 0.2817
Epoch 3/20
1006/1006 [==============================] - 16s 16ms/step - loss: 1.3782 - acc: 0.5934 - val_loss: 1.3832 - val_acc: 0.2488
Epoch 4/20
1006/1006 [==============================] - 16s 16ms/step - loss: 1.3751 - acc: 0.5706 - val_loss: 1.3833 - val_acc: 0.2570
Epoch 5/20
1006/1006 [==============================] - 16s 16ms/step - loss: 1.3722 - acc: 0.5815 - val_loss: 1.3837 - val_acc: 0.2521
Epoch 6/20
1006/1006 [==============================] - 16s 16ms/step - loss: 1.3688 - acc: 0.6362 - val_loss: 1.3844 - val_acc: 0.2718
Epoch 7/20
1006/1006 [================

In [15]:
#Score 
scores = model.evaluate(X_test, Y_test, verbose=1) 
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

607/607 [==============================] - 6s 11ms/step
Test loss: 1.4088647614986538
Test accuracy: 0.2652388793436267


In [17]:
 #precision    recall  f1-score   support
import numpy as np
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(Y_test, y_pred_bool))

607/607 [==============================] - 6s 10ms/step
              precision    recall  f1-score   support

           0       0.25      0.69      0.37       150
           1       0.10      0.01      0.01       150
           2       0.30      0.32      0.31       157
           3       0.38      0.03      0.06       150

    accuracy                           0.27       607
   macro avg       0.26      0.26      0.19       607
weighted avg       0.26      0.27      0.19       607

